In [1]:
%matplotlib inline

**Recuperando o que foi feito na introdução...**

In [2]:
import pandas as pd
import numpy as np

from sklearn.cross_validation import train_test_split

# Lendo csv
csv_data = pd.read_csv("data/spambase.csv")

# Copiando os dados do csv
data = csv_data.values.copy()

# Embaralhando os dados para garantir aleatoriedade entre as amostras
# np.random.shuffle(data)

# Separando atributos de classes
x = data[:, :-1]  # x tem apenas valores entre a primeira e penúltima coluna
y = data[:, -1]  # y tem os valores da última coluna

# 70% dos dados serão utilizados para treinamento e 30% para o teste
# A divisão será estratificada, serão mantidas as proporções de spam e não spam em cada grupo
x_treino, x_teste, y_treino, y_teste = train_test_split(x, y, train_size=0.7, stratify=y)

___
# 2. Utilizando o Classificador Naive Bayes

Já aprendemos na aula teórica como funciona Naive Bayes. Só relembrando: ele se chama "Naive" pois assume, de forma ingênua assumi, que todos os atributos são variáveis independentes, ou seja, são variáveis em que a presença ou valor não está condicionado à existência ou valor de uma outra variável.

Para a nossa base de emails vamos experimentar dois tipos de classificares Naive Bayes disponível no scikit-learn, são eles:

- o Bernoulli Naive Bayes, uma implementação do Naive Bayes em que o conhecimento a priori se resume a existência ou não do atributo.
- Multinomial, uma implementação do Naive Bayes em que o conhecimento a priori se está relacionado com a frequência com que a variável ocorre na base de dados.

A seguir vamos explorar melhor de cada um deles.

##  2.1 Aplicando o Bernoulli Naive Bayes

Nesse classificador, os atributos serão convertidos em valores binários (existe ou não existe), por padrão o threshold da binarização é 0.0, mas um novo *threshold* poderá ser definido pela variável *binarize*.

Vale lembrar que os últimos 3 atributos são sempre valorados (como vimos no gráfico com a distribuição). Isso significa que, após a binarização, os valores desses atributos serão iguais a 1 sempre. Como esses atributos serão iguais para toda as amostras, a presença deles não terá importância para a classificação, logo podem ser descartados -- informação que pode ser útil está sendo jogada fora!

**Para verificar a aplicação desse classificador vamos experimentar alguns cenários:**

1. com todas os atributos;
2. sem os 3 últimos atributos e com o threshold igual a 0;
3. sem os 3 últimos atributos e com o threshold igual ao valor médio de cada um dos atributos.

### 2.1.1 Com todos os atributos

In [3]:
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import classification_report

# Classificando com o Bernoulli Naive Bayes
cls_nb = BernoulliNB()
# Treinamento
cls_nb.fit(x_treino, y_treino)
# Predição
y_nb_pred = cls_nb.predict(x_teste)

print "Bernoulli Naive Bayes"
print classification_report(y_nb_pred, y_teste)

Bernoulli Naive Bayes
             precision    recall  f1-score   support

        0.0       0.92      0.89      0.91       857
        1.0       0.83      0.87      0.85       523

avg / total       0.89      0.88      0.89      1380



### 2.1.2 Sem os 3 últimos atributos e com o threshold igual a 0:

Removendo os atributos:

In [4]:
r_x_treino = np.delete(x_treino, [55, 56, 57], 1)  # removendo os atributos do conjunto de treino
r_x_teste = np.delete(x_teste, [55, 56, 57], 1)  # removendo os atributos do conjunto de teste

Aplicando o classificador:

In [5]:
# Classificando com o Bernoulli Naive Bayes
cls_nb = BernoulliNB(binarize=0.0)  # valor padrão 0.0
# Treinamento
cls_nb.fit(r_x_treino, y_treino)
# Predição
r_y_nb_pred = cls_nb.predict(r_x_teste)

print "Bernoulli Naive Bayes"
print classification_report(r_y_nb_pred, y_teste)

Bernoulli Naive Bayes
             precision    recall  f1-score   support

        0.0       0.92      0.89      0.91       857
        1.0       0.83      0.87      0.85       523

avg / total       0.89      0.88      0.89      1380



**O resultado foi o mesmo do anterior!** De fato os 3 últimos atributos não agregou qualquer valor ao classificador.

### 2.1.2 Sem os 3 últimos atributos e com o threshold igual ao valor médio de cada atributo.


In [6]:
# valor médio dos atributos:
x_medio = np.mean(x, axis=0)
x_medio = np.delete(x_medio, [55, 56, 57])

# Classificando com o Bernoulli Naive Bayes
cls_nb = BernoulliNB(binarize=x_medio)
# Treinamento
cls_nb.fit(r_x_treino, y_treino)
# Predição
r_y_pred_1 = cls_nb.predict(r_x_teste)

print "Bernoulli Naive Bayes"
print classification_report(r_y_pred_1, y_teste)

Bernoulli Naive Bayes
             precision    recall  f1-score   support

        0.0       0.93      0.91      0.92       858
        1.0       0.85      0.89      0.87       522

avg / total       0.90      0.90      0.90      1380



A binarização utilizando como threshold a média dos valores melhorou o desempenho do classificador.

## 2.2 Aplicando o Multinomial Naive Bayes

Este modelo uiliza da frequência dos atributos para a classificação. O termo frequência é definido como o número de vezes que um determinado termo aparece em um documento. 

Só que nem todos os atributos estão em termos de frequência, como é o caso dos últimos 3 atributos. Para que o classificador funcione como desejado, é preciso fazer alguns ajustes na coleção de dados. Então, vamos fazer dois experimentos, um com todos os atributos e, novamente, removendo os últimos 3 atributos :(


### 2.2.1 Sem qualquer alteração nos dados (não correto)

In [7]:
from sklearn.naive_bayes import MultinomialNB

# Classificando com o gaussian Naive Bayes
cls_mnb = MultinomialNB()
# Treinamento
cls_mnb.fit(x_treino, y_treino)
# Predição
y_mnb_pred = cls_mnb.predict(x_teste)

print "Multinomial Naive Bayes"
print classification_report(y_mnb_pred, y_teste)

Multinomial Naive Bayes
             precision    recall  f1-score   support

        0.0       0.84      0.83      0.83       848
        1.0       0.73      0.75      0.74       532

avg / total       0.80      0.80      0.80      1380



### 2.2.2 Removendo os 3 últimos atributos

In [8]:
# Classificando com o Multinomial Naive Bayes
cls_mnb = MultinomialNB()
# Treinamento
cls_mnb.fit(r_x_treino, y_treino)
# Predição
y_mnb_pred = cls_mnb.predict(r_x_teste)

print "Multinomial Naive Bayes"
print classification_report(y_mnb_pred, y_teste)

Multinomial Naive Bayes
             precision    recall  f1-score   support

        0.0       0.78      0.98      0.87       668
        1.0       0.98      0.75      0.85       712

avg / total       0.88      0.86      0.86      1380



### 2.2.3 Como poderíamos aproveitar esses 3 últimos atributos?

- [54] o comprimento médio das palavras escritas toda com letras maiúsculas.
- [55] o comprimento da maior palavra escritas toda com letras maiúsculas.
- [56] o total de letras em maiúsculo no email.

